In [1]:
import sys
sys.path.insert(0, "../..")

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from MPyDATA_examples.Olesik_et_al_2020.setup import Setup
from MPyDATA_examples.Olesik_et_al_2020.demo_plot_convergence  import plot
from MPyDATA_examples.Olesik_et_al_2020.coordinates import x_id, x_p2, x_p3, x_log_of_pn
from MPyDATA.options import Options
from MPyDATA_examples.Olesik_et_al_2020.simulation import Simulation
from MPyDATA_examples.utils.show_plot import show_plot
from joblib import Parallel, parallel_backend, delayed
from MPyDATA_examples.Olesik_et_al_2020.physics.equilibrium_drop_growth import PdfEvolver
from MPyDATA.arakawa_c.discretisation import discretised_analytical_solution
from MPyDATA_examples.utils.error_norms import L2, GMD

In [3]:
GCs = np.linspace(.15,.85, 7)
nrs = np.linspace(64,256, 7, dtype = int)

In [4]:
grid_layout = x_p2()
psi_coord = x_p2()
setup = Setup()

In [5]:
def analysis(nr, GC, opt):
    setup = Setup(nr = nr, mixing_ratios_g_kg = [1,4])
    options = Options(**opt)
    simulation = Simulation(setup, grid_layout,  psi_coord, options, GC)
    simulation.step(simulation.out_steps[-1])
    t = simulation.out_steps[-1] * simulation.dt
    rh = simulation.rh
    pdf_t = PdfEvolver(setup.pdf, setup.drdt, t)
    analytical = discretised_analytical_solution(
                simulation.rh.magnitude,
                lambda r: pdf_t(r * rh.units).magnitude
            ) * pdf_t(rh[0]).units
    numerical = simulation.n
    loc_of_maximum_num = simulation.r[np.argmax(numerical)]
    loc_of_maximum_anal = simulation.r[np.argmax(analytical)]
    maximum_num = np.max(numerical)
    maximum_anal = np.max(analytical)
    measure_location = (loc_of_maximum_num / loc_of_maximum_anal).magnitude
    measure_height = (maximum_num / maximum_anal).magnitude
    error_L2 = L2(numerical.magnitude, analytical.magnitude, simulation.out_steps[-1])
    error_GMD = np.log(GMD(numerical.magnitude, analytical.magnitude, t.magnitude)) / np.log(2)
    return nr, GC, error_GMD, measure_location, measure_height

In [6]:
opt_set = (
    {'n_iters': 1},
     {'n_iters': 2},
     {'n_iters': 2,'infinite_gauge':True},
     {'n_iters': 2, 'infinite_gauge': True, 'flux_corrected_transport': True},
     {'n_iters': 2, 'third_order_terms':True},
     {'n_iters': 3},
     {'n_iters': 3, 'third_order_terms': True, 'infinite_gauge': True, 'flux_corrected_transport': True}
)


In [7]:
for opt in opt_set:
    with parallel_backend('threading', n_jobs=-2):
        results0 = Parallel(verbose=10)(
            delayed(analysis)(nr, GC, opt)
            for nr in nrs
            for GC in GCs
        )
    results = [list(i) for i in zip(*results0)]
    plot_setup = np.array(results[0:2])
    measures = {'log$_2$(err)':results[2], 'location ratio':results[3], 'height ratio':results[4]}
    for measure in measures.keys():
        plot(plot_setup[0], plot_setup[1], measures[measure], name = measure, n_levels = 10)
        show_plot(filename = f'convergence_{measure}.pdf')

[Parallel(n_jobs=-2)]: Using backend ThreadingBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done   4 tasks      | elapsed:   52.3s
[Parallel(n_jobs=-2)]: Done  11 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-2)]: Done  18 tasks      | elapsed:  3.0min
[Parallel(n_jobs=-2)]: Done  27 tasks      | elapsed:  4.5min
[Parallel(n_jobs=-2)]: Done  36 tasks      | elapsed:  6.1min
[Parallel(n_jobs=-2)]: Done  41 out of  49 | elapsed:  7.1min remaining:  1.4min
[Parallel(n_jobs=-2)]: Done  46 out of  49 | elapsed:  7.9min remaining:   30.7s
[Parallel(n_jobs=-2)]: Done  49 out of  49 | elapsed:  8.3min finished
No handles with labels found to put in legend.


../..\MPyDATA_examples\Olesik_et_al_2020\demo_plot_convergence.py:75: UserWarning: No contour levels were found within the data range.
  fig.gca().contour(xi, yi, zi, levels, linewidths=1, colors='k')
No handles with labels found to put in legend.


No handles with labels found to put in legend.


[Parallel(n_jobs=-2)]: Using backend ThreadingBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done   4 tasks      | elapsed:   59.1s
[Parallel(n_jobs=-2)]: Done  11 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-2)]: Done  18 tasks      | elapsed:  3.5min
[Parallel(n_jobs=-2)]: Done  27 tasks      | elapsed:  5.3min
[Parallel(n_jobs=-2)]: Done  36 tasks      | elapsed:  7.2min
[Parallel(n_jobs=-2)]: Done  41 out of  49 | elapsed:  8.3min remaining:  1.6min
[Parallel(n_jobs=-2)]: Done  46 out of  49 | elapsed:  9.2min remaining:   35.8s
[Parallel(n_jobs=-2)]: Done  49 out of  49 | elapsed:  9.7min finished
No handles with labels found to put in legend.


../..\MPyDATA_examples\Olesik_et_al_2020\demo_plot_convergence.py:75: UserWarning: No contour levels were found within the data range.
  fig.gca().contour(xi, yi, zi, levels, linewidths=1, colors='k')
No handles with labels found to put in legend.


...

../..\MPyDATA_examples\Olesik_et_al_2020\demo_plot_convergence.py:75: UserWarning: No contour levels were found within the data range.
  fig.gca().contour(xi, yi, zi, levels, linewidths=1, colors='k')
No handles with labels found to put in legend.


No handles with labels found to put in legend.


[Parallel(n_jobs=-2)]: Using backend ThreadingBackend with 7 concurrent workers.
../..\MPyDATA_examples\Olesik_et_al_2020\setup.py:49: IntegrationWarning: The integral is probably divergent, or slowly convergent.
  )[0] * yunit * xunit ** 4
[Parallel(n_jobs=-2)]: Done   4 tasks      | elapsed:  4.2min
[Parallel(n_jobs=-2)]: Done  11 tasks      | elapsed: 10.8min
[Parallel(n_jobs=-2)]: Done  18 tasks      | elapsed: 17.2min
[Parallel(n_jobs=-2)]: Done  27 tasks      | elapsed: 25.3min
[Parallel(n_jobs=-2)]: Done  36 tasks      | elapsed: 33.9min
[Parallel(n_jobs=-2)]: Done  41 out of  49 | elapsed: 38.6min remaining:  7.5min
[Parallel(n_jobs=-2)]: Done  46 out of  49 | elapsed: 43.1min remaining:  2.8min
[Parallel(n_jobs=-2)]: Done  49 out of  49 | elapsed: 45.7min finished
No handles with labels found to put in legend.


../..\MPyDATA_examples\Olesik_et_al_2020\demo_plot_convergence.py:75: UserWarning: No contour levels were found within the data range.
  fig.gca().contour(xi, yi, zi, levels, linewidths=1, colors='k')
No handles with labels found to put in legend.


../..\MPyDATA_examples\Olesik_et_al_2020\demo_plot_convergence.py:75: UserWarning: No contour levels were found within the data range.
  fig.gca().contour(xi, yi, zi, levels, linewidths=1, colors='k')
No handles with labels found to put in legend.
